In [65]:
%matplotlib inline
import xmitgcm
import xgcm
import xarray as xr
import matplotlib.pyplot as plt
from xlayers.core import layers_apply
import numpy as np
from MITgcmutils import jmd95
import time as time
import matplotlib.colors as colors
from pathlib import Path
import pandas as pd
from scipy.interpolate import interp1d

In [79]:
#load mitgcm output
#datadir='/Users/boerzhang/Desktop/ocean_project/restart_6x16_run_134308800_KGM_KGM2xonly_diag_testdiv_gTracer'
#datadir='/Users/boerzhang/Desktop/ocean_project/restart_6x16_run_134308800_KGM_KGM2xonly_diag_testdiv_GMadv_nobvp'
prefixes=['Stave','Ttave','vVeltave','uVeltave','wVeltave','DIAG_tracer_adv_diff','PTRtave01','GMflux','Tdiftave','GM_Kwz-T','divergence','GMRediTensor']
#prefixes=['GMRediTensor', 'Tdiftave', 'GMflux', 'DIAG_tracer_adv_diff', 'wVeltave', 'Ttave', 'GM_Kwz-T', 'vVeltave', 'PTRtave01', 'Stave', 'uVeltave','La']
prefixes= ['vVeltave', 'DIAG_tracer_adv_diff', 'GM_Kwz-T', 'PTRtave01', 'wVeltave', 'GMflux', 'Ttave', 'Stave', 'Tdiftave', 'GMRediTensor', 'uVeltave','LaVH1RHO','LaHs1RHO']
datadirGM2x='/Users/anthonymeza/Documents/GitHub/test_age_framework/Single_basin_simulations/restart_6x16_run_134308800_KGM_KGM2xonly_diag_testdiv_gTracer_skewflux_modGMdiagnose'
datadirctr='/Users/anthonymeza/Documents/GitHub/test_age_framework/Single_basin_simulations/control_6x16_run_98893440_diag_Jan12_50year_ModGMdiag'
datadirkappa2x='/Users/anthonymeza/Documents/GitHub/test_age_framework/Single_basin_simulations/kappa2x_6x16_run_164090878_diag_Jan12_50year_ModGMdiag'
datadirRedi2x='/Users/anthonymeza/Documents/GitHub/test_age_framework/Single_basin_simulations/Redi2x_6x16_run_135172800_diag_Jan12_50year_ModGMdiag'

#iters=[134308848]
iters='all'

def convert_coords(dataarray, dtype):
    new_coords = {coord: dataarray.coords[coord].astype(dtype) for coord in ["Z", "XC", "YC"]}
    return dataarray.assign_coords(**new_coords)

dsGM2x=xmitgcm.open_mdsdataset(datadirGM2x,prefix=prefixes,iters=iters).astype('float64').compute()
dsGM2x = convert_coords(dsGM2x, "float64")
dsctr=xmitgcm.open_mdsdataset(datadirctr,prefix=prefixes,iters=iters).astype('float64').compute()
dsctr = convert_coords(dsctr, "float64")

dsRedi2x=xmitgcm.open_mdsdataset(datadirRedi2x,prefix=prefixes,iters=iters).astype('float64').compute()
dsRedi2x = convert_coords(dsRedi2x, "float64")

dskappa2x=xmitgcm.open_mdsdataset(datadirkappa2x,prefix=prefixes,iters=iters).astype('float64').compute()
dskappa2x = convert_coords(dskappa2x, "float64")


In [81]:
def replace_negative_age(ds):
    ds["AGE"] = ds.AGE.where(ds.AGE >= 0)
    ds = ds.assign_coords(Z=-ds.Z.values)
    ds.interpolate_na(dim = ["Z", "XC", "YC"])
    ds = ds.assign_coords(Z=-ds.Z.values)
    return ds

In [ ]:
ds_list = [dsctr, dsGM2x, dsRedi2x, dskappa2x]
fnames = ["../TMI_data/single_basin_ctrl.nc", "../TMI_data/single_basin_GM2X.nc", 
          "../TMI_data/single_basin_Redi2X.nc", "../TMI_data/single_basin_Kappa2X.nc"]
for (i, ds) in enumerate(ds_list):
    ds = ds[["Ttave", "Stave", "PTRtave01"]].mean("time", keep_attrs = True).isel(Z = slice(1, None))
    ds = ds.rename({"Ttave":"THETA", "Stave":"SALT", "PTRtave01":"AGE"})
    ds.AGE.values = ds.AGE.values /31104000 #convert age to years
    ds = replace_negative_age(dsctr) #replace negative ages
    ds.AGE.values[0, :, :] = 0 * ds.AGE.values[0, :, :] #remove top level
    ds.to_netcdf(fnames[i], engine = "netcdf4")